https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#sphx-glr-download-tutorial-10-key-features-005-visualization-py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pandas as pd
from pathlib import Path

import optuna

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline



In [3]:
from reprpo.training import train
import tyro
from reprpo.experiments import experiment_configs
from reprpo.hp.space import search_spaces

from reprpo.interventions import Interventions, DPOConfig, ReprPOConfig, ProjGradConfig
from reprpo.interventions.losses import Losses
from reprpo.interventions.transforms import Transforms

## Objective

In [4]:
SEED=42
key_metric = "acc_gain_vs_ref/oos"

torch.manual_seed(SEED)
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
# silence please
import os
from loguru import logger
logger.remove()
logger.remove()
logger.add(os.sys.stderr, level="WARNING")

os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TQDM_DISABLE"] = "true"

In [6]:
f_db = f"sqlite:///../outputs/optuna/optuna.db"
f = f_db.replace('sqlite:///', './')
print(f)
Path(f).parent.mkdir(parents=True, exist_ok=True)
f_db

./../outputs/optuna/optuna.db


'sqlite:///../outputs/optuna/optuna.db'

In [7]:
from reprpo.hp.target import override, tuner_kwargs
from reprpo.experiments import experiment_configs
import copy
import wandb

import optuna.pruners
from optuna_integration.wandb import WeightsAndBiasesCallback

ImportError: cannot import name 'tuner_kwargs' from 'reprpo.hp.target' (/workspace/repr-preference-optimization/reprpo/hp/target.py)

Note on pruning. It's only really usefull with validation metrics and for long jobs over many epochs. I've got a small proxy job so there is no need.

In [ ]:
MAX_TRIALS= 150

def list2tuples(d):
    for k, v in d.items():
        if isinstance(v, list):
            d[k] = tuple(v)
    return d

for starter_experiment_name, trial2args in search_spaces.items():
    study_name = f"{starter_experiment_name}"

    def objective_func(kwargs, trial):
        cfg = copy.deepcopy(experiment_configs[starter_experiment_name][1])
        override(cfg, tuner_kwargs)
        
        override(cfg, kwargs)
        kwargs = list2tuples(kwargs)
        r = train(cfg, trial=trial)
        return r

    def objective(trial: optuna.Trial) -> float:
        kwargs = trial2args(trial)
        r = objective_func(kwargs, trial)
        return r[key_metric]

    os.environ["WANDB_NOTEBOOK_NAME"] = f"{study_name}.ipynb"
    wandb_kwargs = {"project": "reprpo-optuna", "name": study_name}
    wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

    study = optuna.create_study(
        study_name=study_name,
        direction="maximize",
        load_if_exists=True,
        storage=f_db,
        sampler=optuna.samplers.TPESampler(seed=SEED),
        pruner=optuna.pruners.NopPruner(),
    )

    n = 0

    if len(study.trials)>0:
        df = study.trials_dataframe().query('state == "COMPLETE"').sort_values('value', ascending=False)
        n = len(df)

        print(f"loaded {n} {study_name} trials")
    if n < MAX_TRIALS:
        study.optimize(objective, 
                    n_trials=MAX_TRIALS, 
                    callbacks=[wandbc], gc_after_trial=True, 
                    # catch=(Exception,)
                    )

    print('study.best_trial', study.best_trial)

    wandb.finish(quiet=True)

In [ ]:
%debug

In [22]:
# plot_timeline(study)

In [ ]:
plot_optimization_history(study)

In [16]:
# plot_intermediate_values(study)

In [21]:
# plot_contour(study)


In [ ]:
plot_slice(study)


In [ ]:
plot_param_importances(study)

### Apendix 1: dataclass 2 optuna

In [20]:
# import inspect
# import typing
# from typing import Literal

# def optuna_suggest_from_dataclass(t):
#     n = t.__name__
#     print(f'## {n}')
#     sig = inspect.signature(t)
#     for name, param in sig.parameters.items():
#         if param.annotation== bool:
#             print(f'"{name}": trial.suggest_categorical("{name}", [True, False]),')
#         elif param.annotation==int:
#             print(f'"{name}": trial.suggest_int("{name}", 1, 10),')
#         elif param.annotation ==float:
#             print(f'"{name}": trial.suggest_float("{name}", 0.1, 10.0),')
#         elif param.annotation == str:
#             print(f'"{name}": trial.suggest_categorical("{name}", ["a", "b", "c"]),')
#         elif param.annotation == tuple:
#             print(f'"{name}": trial.suggest_categorical("{name}", [(1, 2), (3, 4), (5, 6)]),')
#         elif typing.get_origin(param.annotation) == Literal:
#             print(f'"{name}": trial.suggest_categorical("{name}", {param.annotation.__args__}),')
#         else:
#             print(f"!!Unknown type {param}")
#             # print(name, param.default, param.annotation)

# optuna_suggest_from_dataclass(ReprPOConfig)
# for t in Transforms:
#     print(f'## {t}')
#     optuna_suggest_from_dataclass(t.value)
# for l in Losses:
#     print(f'## {l}')
#     optuna_suggest_from_dataclass(l.value)


# optuna_suggest_from_dataclass(DPOProjGradConfig)